In [16]:
import pandas as pd

#Importo el archivo de GAMES
games=pd.read_csv("Archivos_MVP/games_mvp.csv",encoding="latin-1")

#Importo el archivo de REVIEWS
reviews= pd.read_parquet("Archivos_MVP/reviews_mvp.parquet")
sentimientos=pd.read_parquet("Archivos_MVP/sentimientos_mvp.parquet")

#Importo el archivo de ITEMS
items=pd.read_parquet("Archivos_MVP/item_mvp.parquet")
#DataFrame joineado para funciones 1) y 2). Voy a unir games con items

#Join los dfs: "items" con "games"
#En la tabla items tengo la cantidad de horas jugadas para cada usuario por cada juego (item)
#Asocio a esta tabla el genero del juego
#Las uno mediante item_id 

items['item_id'] = items['item_id'].astype(float) #Necesito igualar el tipo de dato para poder joinear las tablas 
df= pd.merge(items, games, on='item_id', how='inner')
df.tail(3)   #df que voy a utlizar en la funcion

,user_id,item_id,item_name,horas,genres,app_name,date,price
171267,PWNSTARS,32660.0,Delta Force: Xtreme,9.000000,['Action'],Delta Force: Xtreme,2005,19.99
171268,sIuttrak,473460.0,Memory Oblivion Box,9.716667,"['Adventure', 'Casual', 'Indie']",Memory Oblivion Box,2016,9.99
171269,Steamified,333100.0,Demesne,0.366667,"['Adventure', 'Indie', 'Simulation', 'Early Ac...",Demesne,2016,19.99


In [17]:
#FUNCION 1:  PlayTimeGenre
# 1) Paso toda al columna de generos a minuscula 
df['genres'] = df['genres'].str.lower()
#df['genres'].tail(5)

# 2) Filtro los valores que contienen el genero que me interesa
genero="casual"
df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 
#df_filtrado['genres'].tail(5)
##Me devuelve los valores booleanos True que cumplen la condicion del filtro.


#Ya seleccione los datos  que necesito. Me falta agruparlos y sumar las horas de juego por año.
#3) Agrupo los valores por año en cada caso.

df_ordenado=df_filtrado[["date","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)

#df_ordenado.head(10)

df_ordenado.reset_index(inplace=True) #Agrego el indice
df_ordenado.head(10)


#df_filtrado["date"].value_counts()

#df_ordenado['horas'] = df_ordenado['horas'].astype(int)

df_ordenado.head(100)

#print(int(df_ordenado["horas"].iloc[0]))
#print(df_ordenado["date"].iloc[0])


#print(f"Año de lanzamiento con más horas jugadas para género: {genero} --> {df_ordenado.date.iloc[0]}")
print(f"Cantidad de horas jugadas en dicho año: {int(df_ordenado.horas.iloc[0])}")

Cantidad de horas jugadas en dicho año: 10588


In [18]:
#Resumen de Funcion 1: 

# 1) Paso toda al columna de generos a minuscula 
df['genres'] = df['genres'].str.lower()

# 2) Filtro los valores que contienen el genero que me interesa
genero="casual" # a modo prueba
genero=genero.lower()
df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 

#3) Agrupo los valores por año.
df_ordenado=df_filtrado[["date","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)

#4) Reseteo indice numerico para ver las posiciones
df_ordenado.reset_index(inplace=True)
df_ordenado.head(100)

# 4) Imprimo resultado de funcion
print(f"Año de lanzamiento con más horas jugadas para género {genero.upper()}: {df_ordenado.date.iloc[0]}")
print(f"Cantidad de horas jugadas en dicho año: {int(df_ordenado.horas.iloc[0])}")

Año de lanzamiento con más horas jugadas para género CASUAL: 2012
Cantidad de horas jugadas en dicho año: 10588


In [19]:
#Primera Funcion

def PlayTimeGenre( genero : str ):
    genero=genero.lower() #lo paso a minusculas por si esta escrito con mayusculas
    # 1) Paso toda al columna de generos a minuscula
    df['genres'] = df['genres'].str.lower()
    
    # 2) Filtro los valores que contienen el genero que me interesa
    #genero="action" # a modo prueba
    df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 
    
    #3) Agrupo los valores por año.
    df_ordenado=df_filtrado[["date","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)
    
    #4) Reseteo con indices numericos
    df_ordenado.reset_index(inplace=True) #Agrego el indice para ver las posiciones
    
    #5) Imprimo resultado de funcion
    
    return print(f"Año con más horas jugadas para dicho género {genero.upper()}: {df_ordenado.date.iloc[0]}")



PlayTimeGenre("strategy")

Año con más horas jugadas para dicho género STRATEGY: 2012


In [20]:
#FUNCION 2:  UserForGenre
# 1) Paso toda la columna de generos a minuscula 
df['genres'] = df['genres'].str.lower()
genero=genero.lower() #paso a minuscula en caso de que no este de la misma forma

# 2) Filtro los valores que contienen el genero que me interesa
#genero="action"
df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 
df_filtrado['genres'].tail(5)   #Me devuelve los valores booleanos True que cumplen la condicion del filtro.

#3) Creo un nuevo df para ver la cant de horas por jugador/userid
df_ordenado=df_filtrado[["user_id","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)
#df_ordenado.head(5)

# 4)Como ordene de Mayor a Menor, obtengo el primer valor de user_id que corresponde al que mas horas jugo
userid_maxhoras=df_ordenado["user_id"].iloc[0]

print(f"Usuario con mas horas jugadas para género {genero.upper()}: {userid_maxhoras}")

# 5)Renombro la columna que contiene el anio, para evitar confusiones
df_filtrado.rename(columns={'date': 'year'}, inplace=True)   
#df_filtrado.head(5)

# 6)Ya conozco el user id, creo una mascara para quedarme solamente con los datos del user id max horas
df_filtrado = df_filtrado[df_filtrado["user_id"]==userid_maxhoras]
#df_filtrado.head(5)

#5) Me falta mostrar los valores por anio
horas_por_anio = df_filtrado.groupby('year')['horas'].sum()

#6) Creo un dataframe con los valores de la serie que cree
df_horas_por_anio = pd.DataFrame({'Año': horas_por_anio.index, 'Horas': horas_por_anio.values})

#7) Recorro el diccionario para 
for indice, fila in df_horas_por_anio.iterrows():
  año = int(fila['Año'])
  horas_jugadas = int(fila['Horas'])
  print(f"El Año: {año} jugó {horas_jugadas} horas ")

Usuario con mas horas jugadas para género CASUAL: Evilutional
El Año: 2012 jugó 10588 horas 


/var/folders/lh/p6j3glj52sq44ch3v_6_hltc0000gn/T/ipykernel_4338/1943821687.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.rename(columns={'date': 'year'}, inplace=True)


In [21]:
#Segunda Funcion

def UserForGenre( genero : str ):
    # 1) Paso toda la columna de generos a minuscula y el parametro tambien
    df['genres'] = df['genres'].str.lower()
    genero=genero.lower()
    
    # 2) Filtro los valores que contienen el genero que me interesa
    #genero="action"
    df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)]
    
    #3) Creo un nuevo df para ver la cant de horas por jugador/userid
    df_ordenado=df_filtrado[["user_id","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)
    
    # 4)Como ordene de Mayor a Menor, obtengo el primer valor de user_id que corresponde al que mas horas jugo
    userid_maxhoras=df_ordenado["user_id"].iloc[0]
    print(f"Usuario con mas horas jugadas para género {genero.upper()}: {userid_maxhoras} \nHoras jugadas:")
    
    # 5)Renombro la columna que contiene el anio, para evitar confusiones
    df_filtrado.rename(columns={'date': 'year'}, inplace=True)
    
    # 6)Ya conozco el user id, creo una mascara para quedarme solamente con los datos del user id max horas
    df_filtrado = df_filtrado[df_filtrado["user_id"]==userid_maxhoras]
    
    #5) Agrupo las horas por anio
    horas_por_anio = df_filtrado.groupby('year')['horas'].sum()
    
    #6) Creo un dataframe con los valores de la serie que cree
    df_horas_por_anio = pd.DataFrame({'Año': horas_por_anio.index, 'Horas': horas_por_anio.values})
    
    #7) Recorro el diccionario para 
    for indice, fila in df_horas_por_anio.iterrows():
        año = int(fila['Año'])
        horas_jugadas = int(fila['Horas'])
        print(f"Año: {año} -->  {horas_jugadas} horas ")
        
    return

UserForGenre("Action")

Usuario con mas horas jugadas para género ACTION: Evilutional 
Horas jugadas:
Año: 2012 -->  10641 horas 
Año: 2017 -->  175 horas 


/var/folders/lh/p6j3glj52sq44ch3v_6_hltc0000gn/T/ipykernel_4338/2099186033.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.rename(columns={'date': 'year'}, inplace=True)


In [22]:
#Tercera Funcion

#Imprimo df de sentimientos en base a reviews
sentimientos.head(3)

,user_id,posted,item_id,sent_analisis
0,76561198009177886,2014.0,10500,Negativo
1,76561198066371009,2014.0,221640,Positivo
2,rexsniper,2013.0,219740,Neutral


In [23]:
games.head(3)

,genres,app_name,date,price,item_id
0,"['Indie', 'RPG', 'Strategy']",Fantasy Grounds - Mini-Dungeon #004: Summoner'...,2016,2.99,547100
1,"['Action', 'Indie']",Press X to Not Die - Soundtrack,2016,0.99,491410
2,"['Action', 'Adventure', 'Indie', 'RPG']",Light Apprentice - The Comic Book RPG,2017,9.99,569570


In [24]:
reviews.head(3)

,user_id,posted,item_id,review
0,russelmao,NaN,253710,would rather spend $95 preordering overwatch.
1,UncleTobsy,2015.0,730,A Great follow up to an awesome series. Ez mon...
2,76561198090302241,2014.0,304930,when dayz and minecraft have sex they make unt...


In [48]:
#Tercera Funcion
import pandas as pd

sentimientos=pd.read_parquet("Archivos_MVP/sentimientos_mvp.parquet")
sentimientos=pd.DataFrame(sentimientos)
sentimientos.head(3)

def UsersRecommend( año : int ):
    
    #Necesito igualar el tipo de dato para poder joinear las tablas 
    sentimientos['item_id'] = sentimientos['item_id'].astype(float) 
    #Junto las tablas
    df_recomendaciones = pd.merge(sentimientos, games, on='item_id', how='inner')
    #Filtro la data por el año
    df_recomendaciones=df_recomendaciones[df_recomendaciones['date']==año]
    #Filtro la data por recomendaciones positivas y neutrales
    recomendaciones_positivas = df_recomendaciones[df_recomendaciones['sent_analisis'].isin(['Positivo', 'Neutral'])]
    #Agrupo las recomendaciones por juego y cuento los valores que hay en sent_analis
    conteo_sentimientos = recomendaciones_positivas.groupby('app_name')['sent_analisis'].value_counts().unstack(fill_value=0)
    #Sumo el total de recomendaciones
    conteo_sentimientos["cantidad_recomendaciones"] = conteo_sentimientos["Neutral"]+conteo_sentimientos["Positivo"]
    conteo_sentimientos.tail(5)
    #Elimino columnas innecesarias
    conteo_sentimientos=conteo_sentimientos.drop(columns=["Neutral","Positivo"])
    #Ordeno por cantidad de recomnedaciones
    conteo_sentimientos=conteo_sentimientos.sort_values(by="cantidad_recomendaciones",ascending=False)
    conteo_sentimientos=conteo_sentimientos.head(3)
    conteo_sentimientos.head(3)
    
    return conteo_sentimientos


UsersRecommend(2011)

sent_analisis,cantidad_recomendaciones
app_name,
PAYDAYâ¢ The Heist,10
APB Reloaded,7
Magicka,7


In [61]:
#Cuarta funcion
import pandas as pd

sentimientos=pd.read_parquet("Archivos_MVP/sentimientos_mvp.parquet")
sentimientos=pd.DataFrame(sentimientos)
sentimientos.head(3)

def UsersNotRecommend( año : int ): 
    
    #Necesito igualar el tipo de dato para poder joinear las tablas 
    sentimientos['item_id'] = sentimientos['item_id'].astype(float) 
    #Junto las tablas
    df_recomendaciones = pd.merge(sentimientos, games, on='item_id', how='inner')
    #Filtro la data por el año
    df_recomendaciones=df_recomendaciones[df_recomendaciones['date']==2014]
    #Filtro la data por recomendaciones positivas y neutrales
    recomendaciones_negativas = df_recomendaciones[df_recomendaciones['sent_analisis'].isin('Negativo')]
    #Agrupo las recomendaciones por juego y cuento los valores que hay en sent_analis
    conteo_sentimientos = recomendaciones_negativas.groupby('app_name')['sent_analisis'].value_counts().unstack(fill_value=0)
    #Agrupo las recomendaciones por juego y cuento los valores que hay en sent_analis
    conteo_sentimientos = recomendaciones_negativas.groupby('app_name')['sent_analisis'].value_counts().unstack(fill_value=0)
    #Sumo el total de recomendaciones
    conteo_sentimientos["cantidad_recomendaciones"] = conteo_sentimientos["Negativo"]
    conteo_sentimientos.tail(5)
    #Elimino columnas innecesarias
    conteo_sentimientos=conteo_sentimientos.drop(columns=["Neutral","Positivo"])
    #Ordeno por cantidad de recomnedaciones
    conteo_sentimientos=conteo_sentimientos.sort_values(by="cantidad_recomendaciones",ascending=False)
    conteo_sentimientos=conteo_sentimientos.head(3)
    conteo_sentimientos.head(3)
    
    return print(conteo_sentimientos)

